<a href="https://colab.research.google.com/github/elidonda2-web/Alisia-7B-it-/blob/main/Master_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
import os, re
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    # Do this only in Colab notebooks! Otherwise use pip install unsloth
    import torch; v = re.match(r"[0-9\.]{3,}", str(torch.__version__)).group(0)
    xformers = "xformers==" + ("0.0.32.post2" if v == "2.8.0" else "0.0.29.post3")
    !pip install --no-deps bitsandbytes accelerate {xformers} peft trl triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf "datasets>=3.4.1,<4.0.0" "huggingface_hub>=0.34.0" hf_transfer
    !pip install --no-deps unsloth
!pip install transformers==4.56.2
!pip install --no-deps trl==0.22.2

#l'acces token que tu avais copier, colle sa dans login("sa doit etre à l'intérieur")

In [ ]:
from huggingface_hub import login
login("")

In [ ]:
# 📦 Installation des dépendances
#!pip install -q unsloth torch transformers trl datasets accelerate bitsandbytes
!pip install -q gdown

# 🔧 Importations et configuration
import os
import json
import torch
import gdown
import shutil
from datetime import datetime
from datasets import load_dataset, concatenate_datasets
from google.colab import drive
from unsloth import FastLanguageModel
from trl import SFTConfig, SFTTrainer
from transformers import DataCollatorForSeq2Seq

# =======================
# 🔹 Configuration Principale
# =======================
class Config:
    # Modèle et données
    BASE_MODEL = "Gems234/Alisia-7B-it-V1.0"
    DATASET_NAME = "HuggingFaceH4/ultrachat_200k"

    # Objectifs d'entraînement
    TOTAL_DATASET_SIZE = 50000
    TARGET_COVERAGE = 20000  # 20% de 100k
    EXAMPLES_PER_SESSION = 336  # 84 steps × 4 grad_accum

    # Paramètres techniques
    MAX_SEQ_LENGTH = 2048
    LOAD_IN_4BIT = True
    DTYPE = None
    PER_DEVICE_BATCH = 2
    GRAD_ACCUM = 4
    MAX_STEPS = 84
    LEARNING_RATE = 2e-4
    SEED = 3407

    # Chemins
    WORK_DIR = "/content/alisia_collab"
    DRIVE_MOUNT = "/content/drive"
    DRIVE_BASE_PATH = "/content/drive/MyDrive/Alisia_Collab"
    SOURCES_FILE = "sources.json"
    LOCK_TIMEOUT_HOURS = 6

    # === SOURCES COLLABORATIVES COMPLÈTES ===
    MANUAL_SOURCES = {
        "Élie":"https://drive.google.com/drive/folders/1erhDCDQKrDGpSIZ7KFI82cUSaxJy17aS"
    }

config = Config()

# 📁 Montage Google Drive
def setup_drive():
    """Monte Google Drive et crée la structure de dossiers"""
    print("📁 Montage de Google Drive...")
    drive.mount(config.DRIVE_MOUNT)

    # Créer le dossier de base si nécessaire
    os.makedirs(config.DRIVE_BASE_PATH, exist_ok=True)
    print(f"✅ Drive monté: {config.DRIVE_BASE_PATH}")

setup_drive()

# 👤 Système d'identification amélioré
def get_user_identity():
    """Identifie l'utilisateur avec génération d'ID unique"""
    sources = load_sources()

    print("👤 Système d'identification collaboratif")
    print("1. Nouvel utilisateur")
    print("2. Utilisateur existant")

    choice = input("Choisissez une option (1 ou 2): ").strip()

    if choice == "1":
        # Nouvel utilisateur
        user_name = input("Entrez votre nom: ").strip()
        user_id = generate_user_id(sources)

        # Enregistrer le nouvel utilisateur
        if user_name not in sources["participants"]:
            sources["participants"][user_name] = {
                "user_id": user_id,
                "drive_url": "",
                "added_date": datetime.now().isoformat(),
                "sessions_completed": 0,
                "total_steps": 0,
                "last_activity": datetime.now().isoformat(),
                "performance_score": 0.0
            }
            save_sources(sources)
            print(f"✅ Nouvel utilisateur créé: {user_name} (ID: {user_id})")
        else:
            print("❌ Ce nom existe déjà")
            return get_user_identity()

    else:
        # Utilisateur existant
        user_name = input("Entrez votre nom: ").strip()
        if user_name in sources["participants"]:
            user_id = sources["participants"][user_name]["user_id"]
            print(f"✅ Utilisateur reconnu: {user_name} (ID: {user_id})")
        else:
            print("❌ Utilisateur non trouvé")
            return get_user_identity()

    return user_id, user_name

def generate_user_id(sources):
    """Génère un ID utilisateur unique"""
    existing_ids = [user_data["user_id"] for user_data in sources["participants"].values() if "user_id" in user_data]
    new_id = str(len(existing_ids) + 1)
    while new_id in existing_ids:
        new_id = str(int(new_id) + 1)
    return new_id

user_id, user_name = get_user_identity()

In [ ]:
# 🔗 Gestion améliorée de sources.json
def load_sources():
    """Charge ou initialise le fichier sources.json avec la nouvelle structure"""
    sources_path = os.path.join(config.DRIVE_BASE_PATH, config.SOURCES_FILE)

    if os.path.exists(sources_path):
        with open(sources_path, "r") as f:
            sources = json.load(f)

        # Vérifier la structure et migrer si nécessaire
        sources = migrate_sources_structure(sources)
        return sources
    else:
        # Nouvelle structure avec scores
        initial_sources = {
            "participants": {},
            "metadata": {
                "created": datetime.now().isoformat(),
                "total_sessions": 0,
                "last_training_user": None,
                "best_performer": None,
                "total_collective_steps": 0
            },
            "training_rotation": []
        }
        save_sources(initial_sources)
        print("📁 Fichier sources.json créé avec nouvelle structure")
        return initial_sources

def save_sources(sources):
    """Sauvegarde le fichier sources.json"""
    sources_path = os.path.join(config.DRIVE_BASE_PATH, config.SOURCES_FILE)
    with open(sources_path, "w") as f:
        json.dump(sources, f, indent=2)

def migrate_sources_structure(sources):
    """Migre l'ancienne structure vers la nouvelle"""
    print("🔄 Migration automatique de sources.json...")

    if "participants" in sources:
        for user_name, user_data in sources["participants"].items():
            # Ajouter les champs manquants
            if "user_id" not in user_data:
                user_data["user_id"] = generate_user_id(sources)
                print(f"   ✅ ID généré pour {user_name}: {user_data['user_id']}")
            if "total_steps" not in user_data:
                user_data["total_steps"] = 0
            if "performance_score" not in user_data:
                user_data["performance_score"] = 0.0
            if "last_activity" not in user_data:
                user_data["last_activity"] = datetime.now().isoformat()

    if "metadata" not in sources:
        sources["metadata"] = {
            "created": datetime.now().isoformat(),
            "total_sessions": 0,
            "last_training_user": None,
            "best_performer": None,
            "total_collective_steps": 0
        }

    if "training_rotation" not in sources:
        sources["training_rotation"] = []

    save_sources(sources)
    print("✅ Structure de sources.json migrée")
    return sources

def update_user_performance(user_name, steps_completed, coverage_increase):
    """Met à jour le score de performance de l'utilisateur"""
    sources = load_sources()

    if user_name in sources["participants"]:
        user_data = sources["participants"][user_name]
        user_data["sessions_completed"] = user_data.get("sessions_completed", 0) + 1
        user_data["total_steps"] = user_data.get("total_steps", 0) + steps_completed
        user_data["last_activity"] = datetime.now().isoformat()

        # Calcul du score de performance (steps + couverture + régularité)
        performance_score = (
            user_data["total_steps"] * 0.4 +  # Poids pour le volume
            coverage_increase * 100 * 0.3 +   # Poids pour l'efficacité
            user_data["sessions_completed"] * 10 * 0.3  # Poids pour la régularité
        )
        user_data["performance_score"] = round(performance_score, 2)

        # Mettre à jour les métadonnées globales
        sources["metadata"]["total_sessions"] += 1
        sources["metadata"]["last_training_user"] = user_name
        sources["metadata"]["total_collective_steps"] += steps_completed

        # Mettre à jour le meilleur performeur
        current_best = sources["metadata"].get("best_performer")
        if not current_best or user_data["performance_score"] > sources["participants"][current_best]["performance_score"]:
            sources["metadata"]["best_performer"] = user_name

        # Ajouter à la rotation d'entraînement
        if user_name not in sources["training_rotation"]:
            sources["training_rotation"].append(user_name)

        save_sources(sources)
        print(f"📊 Performance mise à jour pour {user_name}: Score {user_data['performance_score']}")

def get_next_trainer():
    """Détermine le prochain formateur basé sur les performances et la rotation"""
    sources = load_sources()

    if not sources["participants"]:
        return user_name  # Premier utilisateur

    # Récupérer tous les utilisateurs actifs
    active_users = [
        name for name, data in sources["participants"].items()
        if data.get("sessions_completed", 0) > 0
    ]

    if not active_users:
        return user_name

    # Stratégie: alterner entre les utilisateurs basé sur la performance récente
    rotation = sources.get("training_rotation", [])

    if not rotation:
        # Premier entraînement, choisir l'utilisateur avec le score le plus bas pour équilibré
        sorted_users = sorted(active_users, key=lambda x: sources["participants"][x]["performance_score"])
        return sorted_users[0]

    # Vérifier le dernier formateur
    last_trainer = sources["metadata"].get("last_training_user")

    if last_trainer == user_name:
        # Même utilisateur que la dernière fois, continuer
        return user_name

    # Trouver le prochain dans la rotation basé sur la performance
    performance_scores = {
        name: data["performance_score"]
        for name, data in sources["participants"].items()
        if name in active_users
    }

    # Donner la priorité aux utilisateurs avec moins de sessions récentes
    if user_name in active_users:
        user_sessions = sources["participants"][user_name]["sessions_completed"]
        avg_sessions = sum(sources["participants"][u]["sessions_completed"] for u in active_users) / len(active_users)

        if user_sessions < avg_sessions:
            return user_name  # Priorité aux nouveaux contributeurs

    # Sinon, choisir basé sur la performance globale
    sorted_by_performance = sorted(performance_scores.items(), key=lambda x: x[1])
    return sorted_by_performance[0][0]  # Utilisateur avec le score le plus bas

def download_drive_folder(drive_url, download_path):
    """Télécharge DIRECTEMENT depuis l'URL Drive"""
    try:
        os.makedirs(download_path, exist_ok=True)
        gdown.download_folder(drive_url, output=download_path, quiet=False)
        return True
    except Exception as e:
        print(f"❌ Erreur téléchargement: {e}")
        return False

# 🔧 Détection Matada Intelligente
def find_matada_file(base_path):
    """Trouve le fichier matada.json dans différentes structures"""
    print(f"🔍 Recherche de matada.json dans: {base_path}")

    if os.path.exists(base_path):
        for root, dirs, files in os.walk(base_path):
            depth = root.replace(base_path, '').count(os.sep)
            if depth > 2:
                continue

            if "matada.json" in files:
                found_path = os.path.join(root, "matada.json")
                print(f"✅ matada.json trouvé: {found_path}")
                return found_path

    print(f"❌ matada.json non trouvé dans: {base_path}")
    return None

def load_matada_smart(source_path):
    """Charge matada.json intelligemment avec fallbacks"""
    print(f"\n📄 Chargement intelligent des métadonnées depuis: {source_path}")

    matada_path = find_matada_file(source_path)

    if matada_path and os.path.exists(matada_path):
        try:
            with open(matada_path, "r") as f:
                matada = json.load(f)
            print(f"✅ matada.json chargé: {matada_path}")
            return matada
        except Exception as e:
            print(f"❌ Erreur lecture {matada_path}: {e}")

    print("⚠️  Utilisation de matada.json par défaut")
    return {
        "covered_examples": [],
        "coverage_percentage": 0.0,
        "total_steps": 0,
        "sessions_completed": 0,
        "derniere_session": {
            "user_name": user_name,
            "user_id": user_id,
            "steps": 0,
            "date": datetime.now().strftime("%Y-%m-%d"),
            "repetitions": 0
        },
        "historique": []
    }

# 🔄 Détection intelligente de la meilleure version
def find_best_source():
    """Trouve la source la plus avancée incluant TOUTES les sources manuelles"""
    sources = load_sources()
    best_source = None
    best_owner = None
    best_total_steps = -1
    best_coverage = 0

    print("🔍 Analyse de TOUTES les sources disponibles...")

    # === COMBINER TOUTES LES SOURCES ===
    all_sources = {}

    # 1. Ajouter les sources du fichier sources.json
    for participant, info in sources["participants"].items():
        all_sources[participant] = info

    # 2. Ajouter TOUTES les sources manuelles du code
    for participant, drive_url in config.MANUAL_SOURCES.items():
        if participant not in all_sources:
            all_sources[participant] = {
                "drive_url": drive_url,
                "user_id": f"manual_{participant}",
                "sessions_completed": 0,
                "total_steps": 0,
                "performance_score": 0.0,
                "added_date": datetime.now().isoformat()
            }
            print(f"   📍 Source manuelle ajoutée: {participant}")
        else:
            # Mettre à jour l'URL si elle a changé
            all_sources[participant]["drive_url"] = drive_url

    print(f"   📊 Total des sources: {len(all_sources)} participants")

    # 3. Recherche sur TOUTES les sources combinées
    for participant, info in all_sources.items():
        drive_url = info.get("drive_url")
        if not drive_url:
            print(f"    ❌ Aucune URL Drive trouvée pour {participant}")
            continue

        participant_path = os.path.join(config.WORK_DIR, "sources", participant)
        print(f"  📂 Vérification de {participant}...")

        if download_drive_folder(drive_url, participant_path):
            matada = load_matada_smart(participant_path)

            if matada and "total_steps" in matada:
                total_steps = matada.get("total_steps", 0)
                coverage = matada.get("coverage_percentage", 0)

                print(f"    📊 {participant}: {total_steps} steps | Couverture: {coverage}%")

                if total_steps > best_total_steps or (total_steps == best_total_steps and coverage > best_coverage):
                    best_total_steps = total_steps
                    best_coverage = coverage
                    best_source = participant_path
                    best_owner = participant
            else:
                print(f"    ❌ Aucun modèle valide trouvé pour {participant}")
        else:
            print(f"    ❌ Impossible de télécharger {participant}")

    if best_source:
        print(f"🎯 MEILLEURE SOURCE: {best_owner} (Steps: {best_total_steps}, Couverture: {best_coverage}%)")
        return best_source, best_owner
    else:
        print("❌ Aucune source valide trouvée")
        return None, None

# 🆕 Initialisation améliorée
def initialize_new_user():
    """Initialise un nouvel utilisateur avec la nouvelle structure"""
    print("\n🆕 Configuration de votre dossier Drive...")

    personal_folder = os.path.join(config.DRIVE_BASE_PATH, user_name)
    os.makedirs(personal_folder, exist_ok=True)

    print("\n📁 COLLEZ VOTRE LIEN DRIVE:")
    print("1. Allez sur votre dossier Drive")
    print("2. Clic droit → 'Partager' → 'Copier le lien'")
    print("3. Collez le lien COMPLET ci-dessous")

    drive_url = input("\n🔗 Collez le lien COMPLET de votre dossier Drive: ").strip()

    # Mettre à jour sources.json avec l'URL
    sources = load_sources()
    if user_name in sources["participants"]:
        sources["participants"][user_name]["drive_url"] = drive_url
        save_sources(sources)

    # Initialiser matada.json
    initial_matada = {
        "total_steps": 0,
        "coverage_percentage": 0.0,
        "covered_examples": [],
        "sessions_completed": 0,
        "derniere_session": {
            "user_name": user_name,
            "user_id": user_id,
            "steps": 0,
            "date": datetime.now().strftime("%Y-%m-%d"),
            "repetitions": 0
        },
        "historique": []
    }

    personal_matada_path = os.path.join(personal_folder, "matada.json")
    with open(personal_matada_path, "w") as f:
        json.dump(initial_matada, f, indent=2)

    personal_lora_path = os.path.join(personal_folder, "lora_model")
    os.makedirs(personal_lora_path, exist_ok=True)

    print(f"✅ Votre dossier personnel est configuré: {personal_folder}")
    return personal_folder

# 🔒 Système de verrou intelligent
def check_lock(source_path):
    """Vérifie et crée un verrou intelligent"""
    lock_path = os.path.join(source_path, "lock.json")

    if os.path.exists(lock_path):
        with open(lock_path, "r") as f:
            lock = json.load(f)

        lock_time = datetime.fromisoformat(lock["since"])
        current_time = datetime.now()
        hours_diff = (current_time - lock_time).total_seconds() / 3600

        if hours_diff < config.LOCK_TIMEOUT_HOURS:
            print(f"🔒 Session verrouillée par {lock['locked_by']} (ID: {lock['user_id']}) depuis {hours_diff:.1f}h")
            return False
        else:
            print("⚠️  Verrou expiré, reprise de contrôle...")

    lock_data = {
        "locked_by": user_name,
        "user_id": user_id,
        "since": datetime.now().isoformat(),
        "session_steps": config.MAX_STEPS
    }

    with open(lock_path, "w") as f:
        json.dump(lock_data, f, indent=2)

    print("🔓 Verrou acquis - session sécurisée")
    return True

def remove_lock(source_path):
    """Supprime le verrou"""
    lock_path = os.path.join(source_path, "lock.json")
    if os.path.exists(lock_path):
        os.remove(lock_path)
        print("🔓 Verrou libéré")

# 📊 Gestion INTELLIGENTE du dataset
def get_dataset_slice(matada):
    """Détermine quelles portions du dataset utiliser pour cette session"""
    sessions_completed = matada.get("sessions_completed", 0)

    start_idx = (sessions_completed * config.EXAMPLES_PER_SESSION) % config.TOTAL_DATASET_SIZE
    end_idx = start_idx + config.EXAMPLES_PER_SESSION

    ranges = []

    if end_idx <= config.TOTAL_DATASET_SIZE:
        ranges.append((start_idx, end_idx))
    else:
        overflow = end_idx - config.TOTAL_DATASET_SIZE
        ranges.append((start_idx, config.TOTAL_DATASET_SIZE))
        ranges.append((0, overflow))
        print("🔄 Recyclage du dataset (wrap-around)")

    covered = set(matada.get("covered_examples", []))
    for s, e in ranges:
        covered.update(range(s, e))

    covered = set(i % config.TOTAL_DATASET_SIZE for i in covered)
    coverage_pct = min(len(covered) / max(1, config.TARGET_COVERAGE) * 100, 100)

    matada["covered_examples"] = sorted(list(covered))
    matada["coverage_percentage"] = round(coverage_pct, 1)

    print(f"📊 Couverture: {len(covered)}/{config.TARGET_COVERAGE} ({coverage_pct:.1f}%)")
    print(f"🔸 Sessions complétées: {sessions_completed}")
    print("🔸 Intervalles sélectionnés:", ", ".join([f"{s}→{e}" for s, e in ranges]))

    return ranges

def prepare_dataset(ranges, tokenizer):
    """Charge les tranches de dataset et applique le format de conversation"""
    if not isinstance(ranges, (list, tuple)):
        raise ValueError("❌ 'ranges' doit être une liste de tuples (start, end)")

    loaded_slices = []
    for s, e in ranges:
        slice_expr = f"train_sft[{s}:{e}]"
        print(f"📥 Chargement: {slice_expr")
        ds_part = load_dataset(config.DATASET_NAME, split=slice_expr)
        loaded_slices.append(ds_part)

    if len(loaded_slices) == 1:
        dataset = loaded_slices[0]
    else:
        dataset = concatenate_datasets(loaded_slices)
        print(f"🔗 {len(loaded_slices)} tranches concaténées -> {len(dataset)} exemples")

    tokenizer.chat_template = """<|im_start|>system
You are Alisia, a helpful, precise, and knowledgeable assistant created by the Alisia Research Team.<|im_end|>
{% for message in messages %}
<|im_start|>{{ message['role'] }}
{{ message['content'] }}<|im_end|>
{% endfor %}
{% if add_generation_prompt %}<|im_start|>assistant
{% endif %}"""

    def formatting_prompts_func(examples):
        conversations = examples["messages"]
        texts = [
            tokenizer.apply_chat_template(convo, tokenize=False, add_generation_prompt=False)
            for convo in conversations
        ]
        return {"text": texts}

    dataset = dataset.map(formatting_prompts_func, batched=True)
    print(f"✅ Dataset prêt pour entraînement : {len(dataset)} exemples")

    return dataset

# 🔍 Nouvelle fonction : Diagnostic des sources
def check_all_sources():
    """Vérifie l'accessibilité de toutes les sources"""
    print("🔍 Diagnostic des sources collaboratives...")
    print(f"📊 {len(config.MANUAL_SOURCES)} participants dans la liste")

    accessible_count = 0
    models_found = 0

    for participant, drive_url in config.MANUAL_SOURCES.items():
        print(f"\n  🔗 {participant}:")
        print(f"     📎 {drive_url}")

        test_path = os.path.join(config.WORK_DIR, "diagnostic", participant)

        if download_drive_folder(drive_url, test_path):
            accessible_count += 1
            print(f"     ✅ Accessible")

            # Vérifier si le dossier contient un modèle
            matada = load_matada_smart(test_path)
            if matada and matada.get("total_steps", 0) > 0:
                models_found += 1
                print(f"     🤖 Modèle trouvé: {matada.get('total_steps', 0)} steps")
                print(f"     📈 Couverture: {matada.get('coverage_percentage', 0)}%")
            else:
                print(f"     📁 Dossier vide ou sans modèle entraîné")
        else:
            print(f"     ❌ Inaccessible")

        # Nettoyer
        if os.path.exists(test_path):
            shutil.rmtree(test_path)

    print(f"\n📋 RÉSUMÉ DIAGNOSTIC:")
    print(f"   ✅ {accessible_count}/{len(config.MANUAL_SOURCES)} sources accessibles")
    print(f"   🤖 {models_found} modèles entraînés trouvés")

    return accessible_count, models_found

# 🧪 Test des sources (exécuter cette ligne après la cellule)
print("🚀 Configuration collaborative prête!")
check_all_sources()

In [ ]:
# 🧠 Chargement intelligent du modèle
def load_model_and_tokenizer(source_path):
    """Charge le modèle et tokenizer avec reprise intelligente"""
    model_path = config.BASE_MODEL
    lora_path = os.path.join(source_path, "lora_model")

    if os.path.exists(os.path.join(lora_path, "adapter_config.json")):
        print("🔄 Reprise depuis le LoRA existant...")
        model_path = lora_path
    else:
        print("🆕 Démarrage depuis le modèle de base...")

    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name=model_path,
        max_seq_length=config.MAX_SEQ_LENGTH,
        dtype=config.DTYPE,
        load_in_4bit=config.LOAD_IN_4BIT,
    )

    return model, tokenizer, lora_path

def setup_lora(model):
    """Configure l'adaptateur LoRA"""
    model = FastLanguageModel.get_peft_model(
        model,
        r=32,
        target_modules=[
            "q_proj", "k_proj", "v_proj", "o_proj",
            "gate_proj", "up_proj", "down_proj",
        ],
        lora_alpha=32,
        lora_dropout=0,
        bias="none",
        use_gradient_checkpointing="unsloth",
        random_state=config.SEED,
        use_rslora=False,
        loftq_config=None,
    )
    return model

# 🔄 Mise à jour CORRIGÉE des métadonnées
def clean_existing_matada(source_path):
    """Nettoie les métadonnées existantes avec des user_id incorrects"""
    matada_path = find_matada_file(source_path)
    if not matada_path:
        return None

    try:
        with open(matada_path, "r") as f:
            matada = json.load(f)

        cleaned = False
        derniere_session = matada.get("derniere_session", {})
        if "drive.google.com" in str(derniere_session.get("user_id", "")):
            print("🧹 Nettoyage de l'user_id dans dernière_session")
            derniere_session["user_id"] = user_id
            cleaned = True

        history = matada.get("historique", [])
        for session in history:
            if "drive.google.com" in str(session.get("user_id", "")):
                print("🧹 Nettoyage de l'user_id dans l'historique")
                session["user_id"] = user_id
                cleaned = True

        if cleaned:
            with open(matada_path, "w") as f:
                json.dump(matada, f, indent=2)
            print("✅ matada.json nettoyé")

        return matada

    except Exception as e:
        print(f"❌ Erreur nettoyage matada: {e}")
        return None

def update_matada(matada, source_path):
    """Met à jour les métadonnées avec validation - VERSION CORRIGÉE"""
    current_date = datetime.now().strftime("%Y-%m-%d")

    derniere_session = matada.get("derniere_session", {})
    if "drive.google.com" in str(derniere_session.get("user_id", "")):
        print("⚠️  Nettoyage de l'user_id précédent (contient une URL)")
        derniere_session["user_id"] = user_id

    can_compact = (derniere_session.get("user_id") == user_id and
                   derniere_session.get("date") == current_date)

    if can_compact:
        print("🔄 Compaction de session (même jour, même utilisateur)")
        derniere_session["steps"] += config.MAX_STEPS
        derniere_session["repetitions"] = derniere_session.get("repetitions", 0) + 1

        history = matada.get("historique", [])
        for session in history:
            if (session.get("user_id") == user_id and
                session.get("date") == current_date):
                session["steps"] = derniere_session["steps"]
                session["repetitions"] = derniere_session["repetitions"]
                break
    else:
        print("🆕 Nouvelle session détectée")
        new_session = {
            "user_name": user_name,
            "user_id": user_id,
            "steps": config.MAX_STEPS,
            "date": current_date,
            "repetitions": 1
        }
        matada["derniere_session"] = new_session

        history = matada.get("historique", [])
        history.append(new_session)
        matada["historique"] = history[-10:]

    # Calcul de l'augmentation de couverture pour le score de performance
    old_coverage = matada.get("coverage_percentage", 0)
    matada["total_steps"] = matada.get("total_steps", 0) + config.MAX_STEPS
    matada["sessions_completed"] = matada.get("sessions_completed", 0) + 1
    coverage_increase = matada.get("coverage_percentage", 0) - old_coverage

    # Sauvegarde locale
    matada_path = find_matada_file(source_path) or os.path.join(source_path, "matada.json")
    with open(matada_path, "w") as f:
        json.dump(matada, f, indent=2)

    print(f"📈 Métadonnées mises à jour - Total steps: {matada['total_steps']}")
    print(f"📅 Dernière session: {matada['derniere_session']['user_name']} - {matada['derniere_session']['steps']} steps")

    return coverage_increase

# 💾 SAUVEGARDE CORRIGÉE
def save_model_and_tokenizer(model, tokenizer, lora_path):
    """Sauvegarde le modèle et tokenizer"""
    print("💾 Sauvegarde du modèle LoRA...")
    model.save_pretrained(lora_path)
    tokenizer.save_pretrained(lora_path)
    print(f"✅ Modèle sauvegardé dans: {lora_path}")
    print(f"✅ Tokenizer sauvegardé dans: {lora_path}")

def save_to_personal_drive(model, tokenizer, lora_path, matada, source_owner):
    """Sauvegarde les résultats dans le Drive personnel de l'utilisateur"""
    try:
        personal_folder = os.path.join(config.DRIVE_BASE_PATH, user_name)
        os.makedirs(personal_folder, exist_ok=True)

        personal_lora_path = os.path.join(personal_folder, "lora_model")

        if os.path.exists(personal_lora_path):
            shutil.rmtree(personal_lora_path)

        os.makedirs(personal_lora_path, exist_ok=True)

        print("💾 Sauvegarde directe dans votre Drive...")
        model.save_pretrained(personal_lora_path)
        tokenizer.save_pretrained(personal_lora_path)

        print(f"✅ LoRA sauvegardé dans: {personal_lora_path}")

        personal_matada_path = os.path.join(personal_folder, "matada.json")
        with open(personal_matada_path, "w") as f:
            json.dump(matada, f, indent=2)
        print(f"📄 matada.json sauvegardé dans: {personal_matada_path}")

        # Mettre à jour les performances dans sources.json
        coverage_increase = matada.get("coverage_percentage", 0) - load_matada_smart(lora_path).get("coverage_percentage", 0)
        update_user_performance(user_name, config.MAX_STEPS, coverage_increase)

        info_content = f"""Participant: {user_name} (ID: {user_id})
Dernière session: {datetime.now().strftime('%Y-%m-%d %H:%M')}
Source reprise: {source_owner}
Couverture: {matada['coverage_percentage']}%
Total Steps: {matada['total_steps']}
Sessions complétées: {matada['sessions_completed']}
Performance Score: {load_sources()['participants'][user_name]['performance_score']}
"""

        info_path = os.path.join(personal_folder, "info.txt")
        with open(info_path, "w") as f:
            f.write(info_content)

        print(f"✅ Tous les fichiers sauvegardés dans votre Drive personnel: {personal_folder}")
        return personal_folder

    except Exception as e:
        print(f"❌ Erreur sauvegarde Drive personnel: {e}")
        return None

# 🚀 EXÉCUTION PRINCIPALE AMÉLIORÉE
def main():
    print("🚀 Démarrage du système collaboratif Alisia...")
    print(f"👋 Bienvenue {user_name} (ID: {user_id})!")

    # Vérifier si c'est le tour de cet utilisateur
    next_trainer = get_next_trainer()
    if next_trainer != user_name:
        print(f"⏳ Ce n'est pas encore votre tour. Prochain formateur: {next_trainer}")
        print("📊 Scores actuels:")
        sources = load_sources()
        for name, data in sources["participants"].items():
            print(f"   {name}: {data['performance_score']} points")
        return

    print("✅ C'est votre tour de contribuer!")

    print("\n🔍 Recherche de la meilleure source...")
    source_path, source_owner = find_best_source()

    if not source_path:
        print("❌ Aucune source valide trouvée!")
        print("🆕 Création d'une nouvelle session...")

        source_path = initialize_new_user()
        if not source_path:
            print("❌ Échec de l'initialisation")
            return
        source_owner = user_name

        matada_path = os.path.join(source_path, "matada.json")
        with open(matada_path, "r") as f:
            matada = json.load(f)

        print("✅ Nouvelle session créée!")
    else:
        print("🧹 Vérification des métadonnées existantes...")
        matada = clean_existing_matada(source_path) or load_matada_smart(source_path)

    print(f"✅ Source sélectionnée: {source_owner}")

    print("\n🔒 Vérification du verrou...")
    if not check_lock(source_path):
        print("❌ Impossible d'acquérir le verrou - réessayez plus tard")
        return

    try:
        print("\n📊 Calcul de la portion dataset...")
        ranges = get_dataset_slice(matada)

        print("\n🧠 Chargement du modèle...")
        model, tokenizer, lora_path = load_model_and_tokenizer(source_path)
        model = setup_lora(model)

        print("\n📥 Préparation des données...")
        dataset = prepare_dataset(ranges, tokenizer)

        print("\n⚙️ Configuration de l'entraînement...")
        data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer)

        sft_config = SFTConfig(
            per_device_train_batch_size=config.PER_DEVICE_BATCH,
            gradient_accumulation_steps=config.GRAD_ACCUM,
            warmup_steps=5,
            max_steps=config.MAX_STEPS,
            learning_rate=config.LEARNING_RATE,
            logging_steps=1,
            optim="adamw_8bit",
            weight_decay=0.01,
            lr_scheduler_type="linear",
            seed=config.SEED,
            output_dir=lora_path,
            report_to="none"
        )

        trainer = SFTTrainer(
            model=model,
            tokenizer=tokenizer,
            train_dataset=dataset,
            dataset_text_field="text",
            max_seq_length=config.MAX_SEQ_LENGTH,
            data_collator=data_collator,
            packing=False,
            args=sft_config
        )

        print(f"\n🚀 Début de l'entraînement ({config.MAX_STEPS} steps)...")
        trainer_stats = trainer.train()
        print("✅ Entraînement terminé avec succès!")

        print("\n💾 Sauvegarde locale...")
        save_model_and_tokenizer(model, tokenizer, lora_path)

        print("\n📈 Mise à jour des métadonnées...")
        coverage_increase = update_matada(matada, source_path)

        print("\n💾 Sauvegarde dans votre Drive personnel...")
        personal_folder = save_to_personal_drive(model, tokenizer, lora_path, matada, source_owner)

        if personal_folder:
            print(f"\n🎉 Session terminée avec succès!")
            print(f"📊 Progression totale: {matada['coverage_percentage']}%")
            print(f"📈 Total steps cumulés: {matada['total_steps']}")

            sources = load_sources()
            user_data = sources["participants"][user_name]
            print(f"🏆 Votre score de performance: {user_data['performance_score']}")
            print(f"📁 Fichiers sauvegardés dans: {personal_folder}")

            derniere_session = matada.get("derniere_session", {})
            print(f"👤 Votre session: {derniere_session.get('steps', 0)} steps")
            if derniere_session.get('repetitions', 1) > 1:
                print(f"🔄 Sessions compactées aujourd'hui: {derniere_session.get('repetitions', 1)}")

            # Afficher le classement
            print("\n🏅 CLASSEMENT DES CONTRIBUTEURS:")
            sorted_users = sorted(sources["participants"].items(), key=lambda x: x[1]["performance_score"], reverse=True)
            for i, (name, data) in enumerate(sorted_users, 1):
                marker = "👑" if i == 1 else "  "
                print(f"   {marker} {i}. {name}: {data['performance_score']} points ({data['sessions_completed']} sessions)")
        else:
            print("❌ Erreur lors de la sauvegarde dans Drive")

    except Exception as e:
        print(f"❌ Erreur pendant l'exécution: {e}")
        import traceback
        traceback.print_exc()
    finally:
        remove_lock(source_path)

# 🎯 LANCEMENT
if __name__ == "__main__":
    main()

#l'objectif du code est de nous faire à atteindre au moins 20% du dataset utiliser dans le code et sa va fonctionner pour tout les autres datasets.